In [ ]:
import pandas as pd
import re
# URL pública do Google Sheets
sheet_id = "1XuGWm_gDG5edw9YkznTQGABTBah1Ptz9lfstoFdGVbA"
sheet_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"

# Lê diretamente como CSV
dados = pd.read_csv(sheet_url)
print(dados.columns.tolist())

In [ ]:
def limpar_texto(texto):
    texto = texto.lower()
    texto = texto.strip()
    texto = re.sub(r"\s+", " ", texto)
    texto = re.sub(r"[^\w\s,;]", "", texto)
    return texto

def separar_afiliacoes(texto):
    if pd.isna(texto):
        return []
    texto = limpar_texto(texto)
    
    # Tentativa de separar em blocos "afiliação ; país"
    # O padrão espera pelo menos um ";"
    partes = texto.split(";")
    if len(partes) < 2:
        return []
    
    resultado = []
    # Percorre as partes em pares (afiliação, país)
    for i in range(0, len(partes) - 1, 2):
        afil = partes[i].strip()
        pais = partes[i+1].strip()
        
        # Agora separar múltiplas afiliações dentro da afiliação
        # que podem estar separadas por vírgula, "and", "&"
        afil_individuals = re.split(r",| and | & ", afil)
        
        for afil_i in afil_individuals:
            afil_i = afil_i.strip()
            if afil_i:
                resultado.append(f"{afil_i} ; {pais}")
    return resultado

afiliacoes_expandidas = dados["Afiliation"].dropna().apply(separar_afiliacoes).explode()
contagem = afiliacoes_expandidas.value_counts().reset_index()
contagem.columns = ["Afiliacao_e_Pais", "Quantidade"]

print("\n=== Afiliacoes únicas agrupadas e contadas ===\n")
for _, row in contagem.iterrows():
    print(f"{row['Afiliacao_e_Pais']}: {row['Quantidade']}")
    

In [ ]:
# === Contagem por Região ===
contagem_por_regiao = (
    dados[dados["Region"].notna()]
    .groupby("Region")
    .size()
    .reset_index(name="Quantidade")
    .sort_values(by="Quantidade", ascending=False)
)

print("\n=== Contagem de trabalhos por Região ===\n")
print(contagem_por_regiao)


In [ ]:
# === Contagem por País ===
# Lidar com múltiplos países separados por vírgula
def separar_paises(campo):
    if pd.isna(campo):
        return []
    return [pais.strip() for pais in str(campo).split(",")]

# Expandir países em linhas individuais
dados_paises = dados["country"].dropna().apply(separar_paises)
dados_paises_explodido = dados_paises.explode()

contagem_por_pais = (
    dados_paises_explodido.value_counts()
    .reset_index()
    .rename(columns={"index": "country", 0: "Quantidade"})
)

print("\n=== Contagem de trabalhos por País ===\n")
print(contagem_por_pais.to_string(index=False))  # imprime todos os países


In [ ]:
# === Contagem de países por Região ===
dados_regiao_pais = (
    dados[["Region", "country"]]
    .dropna()
    .assign(country=lambda df: df["country"].apply(separar_paises))
    .explode("country")
)

dados_regiao_pais["country"] = dados_regiao_pais["country"].str.strip()

contagem_pais_regiao = (
    dados_regiao_pais
    .groupby(["Region", "country"])
    .size()
    .reset_index(name="Quantidade")
    .sort_values(by=["Region", "Quantidade"], ascending=[True, False])
)

print("\n=== Contagem de países por Região ===\n")
print(contagem_pais_regiao.to_string(index=False))


In [ ]:
# === Contagem por Autor ===
# Lidar com múltiplos autores separados por vírgula
def separar_autores(campo):
    if pd.isna(campo):
        return []
    return [autor.strip() for autor in str(campo).split(",")]

# Expandir autores em linhas individuais
dados_autores = dados["Autores"].dropna().apply(separar_autores)
dados_autores_explodido = dados_autores.explode()

contagem_por_autor = (
    dados_autores_explodido.value_counts()
    .reset_index()
    .rename(columns={"index": "Autor", 0: "Quantidade"})
)

print("\n=== Contagem de trabalhos por Autor ===\n")
print(contagem_por_autor.to_string(index=False))



In [ ]:
referencias_separadas = []

for cell in dados["Ref"]:
    if isinstance(cell, str) and cell.strip():
        # Divide pelas duplas quebras de linha
        refs = cell.split('\n\n')
        for ref in refs:
            ref = ref.strip()
            if ref:
                # Limpa quebras de linha e múltiplos espaços dentro da referência
                ref = ref.replace('\n', ' ').replace('\r', ' ')
                ref = ' '.join(ref.split())
                referencias_separadas.append(ref)

# Cria DataFrame numerado
df_referencias = pd.DataFrame({
    "Referência": referencias_separadas
})

print(df_referencias)


In [ ]:
import pandas as pd
import re

referencias_separadas = []

for cell in dados["Ref"]:
    if isinstance(cell, str) and cell.strip():
        # Normaliza quebras de linha
        temp = cell.replace('\r\n', '\n').replace('\r', '\n')
        # Divide pelas duplas quebras de linha (ou mais)
        refs = re.split(r'\n{2,}', temp)
        
        for ref in refs:
            ref = ref.strip()
            if ref:
                # Remove quebras de linha internas, deixa tudo em uma linha só
                ref = ' '.join(ref.split())
                referencias_separadas.append(ref)

# Cria dataframe
df_separado = pd.DataFrame({"Referência": referencias_separadas})

import re

def extrair_autor_ano(texto):
    m = re.search(r'^(.*?)(\(\d{4}\))', texto)
    if m:
        autores = m.group(1).rstrip('.,; ')
        ano = m.group(2)
        return f"{autores} {ano}"
    return texto



# Aplica função para extrair autor e ano
df_separado['Autor(es) e Ano'] = df_separado['Referência'].apply(extrair_autores_ano)

# Remove linhas sem resultado válido
df_limpo = df_separado.dropna(subset=['Autor(es) e Ano']).reset_index(drop=True)

print(df_limpo[['Autor(es) e Ano']])

